# Alice in Wonderland -- A New Chapter!
In this notebook I'm using LSTM to generate new text based on Alice in Wonderland. Let's see how it goes!

## Data Source
The original text is from project [Gutenberg](https://www.gutenberg.org/files/11/11-0.txt).


In [2]:
def load_data(path):
    """
    Load Dataset from File
    """
    import os
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()

    return data


data_file = 'Alice_in_Wonderland.txt'
text = load_data(data_file)
# Ignore license text and truncate to suitable size
text = text[710:120000]


## Explore the Data


In [3]:
view_sentence_range = (1000, 1050)


import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len(set(text.split()))))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 4604
Number of scenes: 659
Average number of sentences in each scene: 2.1077389984825494
Number of lines: 2048
Average number of words in each line: 10.72119140625

The sentences 1000 to 1050:
   ‘You are old, Father William,’ the young man said,
    ‘And your hair has become very white;
   And yet you incessantly stand on your head--
    Do you think, at your age, it is right?’

   ‘In my youth,’ Father William replied to his son,
    ‘I feared it might injure the brain;
   But, now that I’m perfectly sure I have none,
    Why, I do it again and again.’

   ‘You are old,’ said the youth, ‘as I mentioned before,
    And have grown most uncommonly fat;
   Yet you turned a back-somersault in at the door--
    Pray, what is the reason of that?’

   ‘In my youth,’ said the sage, as he shook his grey locks,
    ‘I kept all my limbs very supple
   By the use of this ointment--one shilling the box--
    Allow me to sell you a couple?’

   ‘You

## Preprocessing Functions
A couple of handy functions to help with running the model later:
- Lookup Table (words to integers and back)
- Tokenize Punctuation


In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    vocab = set(text)
    vocab_to_int = {w:i for i, w in enumerate(vocab, 1)}
    int_to_vocab = {v:k for k, v in vocab_to_int.items()}
    
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
Punctuations like periods and exclamation marks complicate things for the neural network. This function will tokenize symbols like "!" into "||Exclamation_Mark||".


In [215]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    
    punctuation_dict = { '.': '||Period||',
    ',': '||comma||',
    '--': '||dash||',
    '(': '||left_parentheses||',
    ')': '||right_parentheses||',
    ';': '||semicolon||',
    '?': '||question||',
    '\n': '||return||',
    '"': '||double_quote||',
    '!': '||exclamation||'}
    
    return punctuation_dict


tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [143]:
import helper

# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(text, token_lookup, create_lookup_tables)


# Check Point
A quick check point so we don't have to start from the beginning next time.

In [216]:

import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building the Neural Network
To simplify the many tasks of building the NN, we'll implement the following functions:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [223]:

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    print('No GPU found.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
No GPU found.


### Input
The `get_inputs()` function creates the TF Placeholders we need.  

In [218]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    
    Inputs = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learningrate')
    
    
    return (Inputs, Targets, LearningRate)


tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).


In [219]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initial state)
    """
    
    lstm_layers = 1
    
    stacked_rnn = []
    for i in range(lstm_layers):
        stacked_rnn.append(tf.contrib.rnn.BasicLSTMCell(rnn_size))
    
    #lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell(stacked_rnn)
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")

    return (cell, initial_state)


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data`.

In [220]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


tests.test_get_embed(get_embed)

Tests Passed


### Building the RNN
We'll use the cells from `get_init_cell()` to create a dynamic RNN.


In [221]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    
    return (outputs, final_state)


tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Putting all the pieces together, using the functions above, and adding a fully connected layer with linear activation to compute logits.


In [222]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    embed_inputs = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed_inputs)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return (logits, final_state)


tests.test_build_nn(build_nn)

Tests Passed


### Batches
The following function will create batches of inputs and targets using `int_text`.  The batches are a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`


In [224]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    step = batch_size * seq_length
    
    num_of_batches = (len(int_text) - 1) // step
    int_text = int_text[:(num_of_batches * step) + 1] 
    
    # set the last target token to the same value as the first input token
    int_text[len(int_text) - 1] = int_text[0]
  
    # find the inputs
    input_seq = [int_text[i * seq_length : i * seq_length + seq_length] for i in range(0, num_of_batches * batch_size)]
    
    # find the targets, same as inputs but shifted by 1
    int_text = int_text[1:] 
    target_seq = [int_text[i * seq_length : i * seq_length + seq_length] for i in range(0, num_of_batches * batch_size)]
    
    outputs = []
    for n in range(num_of_batches):
        inputs = []
        targets = []
        
        for j in range(batch_size):
            inputs.append(input_seq[j * num_of_batches + n])
            targets.append(target_seq[j * num_of_batches + n])
        outputs.append([inputs, targets])
        
    outputs = np.array(outputs)    
    return outputs
    
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
The following parameters can be tuned as needed:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [154]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 512
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = .01
# Show stats for every n number of batches
show_every_n_batches = 10

save_dir = './save'

### Building the Graph


In [225]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Adam Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Training the Model



In [152]:

batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/2   train_loss = 8.012
Epoch   5 Batch    0/2   train_loss = 5.272
Epoch  10 Batch    0/2   train_loss = 4.458
Epoch  15 Batch    0/2   train_loss = 3.729
Epoch  20 Batch    0/2   train_loss = 3.068
Epoch  25 Batch    0/2   train_loss = 2.520
Epoch  30 Batch    0/2   train_loss = 2.061
Epoch  35 Batch    0/2   train_loss = 1.664
Epoch  40 Batch    0/2   train_loss = 1.325
Epoch  45 Batch    0/2   train_loss = 1.030
Epoch  50 Batch    0/2   train_loss = 0.801
Epoch  55 Batch    0/2   train_loss = 0.620
Epoch  60 Batch    0/2   train_loss = 0.480
Epoch  65 Batch    0/2   train_loss = 0.361
Epoch  70 Batch    0/2   train_loss = 0.277
Epoch  75 Batch    0/2   train_loss = 0.211
Epoch  80 Batch    0/2   train_loss = 0.166
Epoch  85 Batch    0/2   train_loss = 0.133
Epoch  90 Batch    0/2   train_loss = 0.108
Epoch  95 Batch    0/2   train_loss = 0.091
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating new text.

In [226]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [227]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Text Generation Functions
### Get Tensors
This function gets the tensors we need from the graph.


In [228]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor  = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)



tests.test_get_tensors(get_tensors)

Tests Passed


### Generate Next Word
Function to select the next word using `probabilities`.

In [229]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """

    # find the top 5 highest probability words, and pick one at random
    pick_id = np.random.randint(1,2) # adjust randmoness as needed
    highest_prob_index = 0
    for i in range(pick_id):
        highest_prob_index = np.argmax(probabilities)
        probabilities[highest_prob_index] = 0
        
        
    return int_to_vocab[highest_prob_index]

tests.test_pick_word(pick_word)

Tests Passed


## Generate Text
Finally! Let's see if the NN will write Nobel worthy prose.


In [201]:
gen_length = 500 # how many words to generate

prime_word = 'alice' # the word to trigger it all.


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    init_text = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        init_text = init_text.replace(' ' + token.lower(), key)
    init_text = init_text.replace('\n ', '\n')
    init_text = init_text.replace('( ', '(')
        
    print()
    print(init_text)

INFO:tensorflow:Restoring parameters from ./save

alice: ’

‘which is just the case with mine, ’ said the hatter.

alice felt dreadfully puzzled. the hatter’s remark seemed to have no
sort of meaning in it, and yet it was certainly english. ‘i don’t quite
understand you, and listen to me! i’ll soon make you
dry enough! ’ they all sat down at once, in a large ring, with the mouse
in the middle. alice kept tossing
the baby violently up and down, and the poor little thing howled so,
that alice could hardly hear the words:-- ’

she had quite forgotten the duchess by this time, and was a little
startled when she heard her voice close to her ear. ‘you’re thinking
about something, and
perhaps after all it might tell her something worth hearing. for some
minutes it puffed away without speaking, but at last it unfolded its
arms, she
kept fanning herself all the time she went on talking: ‘dear, dear! how
queer everything upon bill! ’ said the hatter.

alice felt dreadfully puzzled. the hatter’s 

In [230]:
# Pick random phrases from the generated text and check against the original text
text.find('alice kept tossing')

-1

# A Little Absurd?
Yeah, but so what. Have you seen the original? It kind of kept the spirit of it!
